# Exploring Toronto

Here, we build a Database with the information on the Neighbourhoods and Boroughs of Toronto. For this, we scrape the information from the <a href=https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M>Wikipedia website with the List of postal codes of Canada starting with the letter M</a>.

We need to import the pandas library:

In [1]:
import pandas as pd

### Getting the dataframe from Wikipedia's website

In [2]:
html_data = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(html_data)[0]

# Renaming Postcode -> Postalcode and Neighbourhood -> Neighborhood
df.rename(columns={'Postcode':'PostalCode', 'Neighbourhood':'Neighborhood'}, inplace=True)

df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


(Note that the dataframe already consists of three columns: PostalCode, Borough, and Neighborhood)

### Preprocessing Dataframe

We now follow the instructions to manipulate the Dataframe:

* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [3]:
# Removing 'Not assigned' boroughs:
df = df[ df['Borough'] != 'Not assigned' ]
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


* More than one neighborhood can exist in one postal code area.  
For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park.  
These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [4]:
# Combining equal postal codes
df_group = df.groupby(['PostalCode','Borough'], as_index=False)['Neighborhood'].agg(lambda x: ', '.join(x))
# Check if PostalCode M5A is correct
df_group[ df_group['PostalCode'] == 'M5A' ]

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.  
So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [5]:
df_group.loc[df_group['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Not assigned


In [6]:
# Copying Borough's name to unassigned Neighborhoods
df_group.loc[df_group['Neighborhood'] == 'Not assigned','Neighborhood'] = df_group[ df_group['Neighborhood'] == 'Not assigned' ]['Borough']

# Checking if there's any unassigned Neighborhood
df_group[ df_group['Neighborhood'] == "Not assigned" ]

,PostalCode,Borough,Neighborhood


In [7]:
# Checking Queen's Park
df_group[ df_group['Neighborhood'] == "Queen's Park" ]

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


In [8]:
df_group.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [9]:
df_group.shape

(103, 3)